In [1]:
import overpy
import shapefile
import csv
import time
import geojson


inshp = "area/reg_buff.shp"
sf = shapefile.Reader(inshp)
bbox = sf.bbox
in_tags = [['social', ["amenity", "shop", "building", "tourism", "leisure", '"landuse"="religious"',
                        '"highway" = "bus_stop"', "government", '"boundary"="protected_area"', '"office"="government"']]
            ]

In [2]:
def convert_int_none(x):
    try:
        if x == None:
            return ''
        else:
            return int(x)
    except ValueError:
        return x

def all_unique(in_list):
    seen = list()
    return not any(l in seen or seen.append(l) for l in in_list)

In [3]:
def get_osm(tag):
    """
    requests data from OSM using Overpass API
    :param tag: OSM tag
    :return: dictionary {'node' : overpy.Result, 'way': overpy.Result, 'rel': [....]}
    """
    query = lambda rel_type: api.query("""({}({},{},{},{}) [{}];);(._;>;);out;"""
                                           .format(rel_type, bbox[1], bbox[0], bbox[3], bbox[2], tag))

    def query_rels(ids):
        while True:
            try:
                return api.query("""relation({});(._;>;);out;""".format(ids))
            except:
                time.sleep(4)

    api = overpy.Overpass()
    result = dict()
    do_queries = [['node', query('node')], ['way', query('way')],
         ['rel', [query_rels(ID) for ID in query('rel').get_relation_ids()]]]
    for q in do_queries:
        while True:
            try:
                result[q[0]] = q[1]
                break
            except:
                time.sleep(4)  # time delay for the next query 3 seconds in case of many requests
    return result

In [4]:
keys = in_tags[0][1]
res = get_osm(keys[0])
for k in keys[1:]:
    print(k)
    ares = get_osm(k)       # {'node' : overpy.Result, 'way': overpy.Result, 'rel': [....]} for each key
    res['node'].expand(ares['node'])
    res['way'].expand(ares['way'])
    res['rel'].extend(ares['rel'])
print('Done')

shop
building
tourism
leisure
"landuse"="religious"
"highway" = "bus_stop"
government
"boundary"="protected_area"
"office"="government"
Done


In [29]:
res['rel'][0].relations.tags

AttributeError: 'list' object has no attribute 'tags'

In [30]:
objs = {'Point': {}, 'LineString': {'coords': [], 'params': []}, 'Polygon': {'coords': [], 'params': []},
            'MultiPolygon': {'coords': [], 'params': []}, 'MultiLineString': {'coords': [], 'params': []}}
#Points
objs['Point']['coords'] = [[float(node.lon), float(node.lat)] for node in res['node'].nodes]
objs['Point']['params'] = [node.tags for node in res['node'].nodes]

In [37]:
res['rel']

In [21]:
for i in rres.relations:
    print(i.id)

4459388


In [14]:
objs['Point']['params'][0]['id'] = 5135360854
objs['Point']['params'][0]

{'amenity': 'fuel', 'id': 5135360854, 'name': 'БРСМ', 'name:ru': 'БРСМ'}